In [158]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [140]:
biogrid = pd.read_csv('BIOGRID-ORGANISM-Homo_sapiens-4.4.232.tab3.txt', sep='\t')


C:\Users\annei\AppData\Local\Temp\ipykernel_14876\1437345336.py:1: DtypeWarning: Columns (1,2,18) have mixed types. Specify dtype option on import or set low_memory=False.
  biogrid = pd.read_csv('BIOGRID-ORGANISM-Homo_sapiens-4.4.232.tab3.txt', sep='\t')


In [153]:
biogrid = biogrid[['Official Symbol Interactor A','Official Symbol Interactor B']]

In [ ]:
biogrid = biogrid[['Official Symbol Interactor A','Official Symbol Interactor B','Score']]
biogrid = biogrid[biogrid['Score'] != '-']
""" Quantitative Score. This will be a positive for negative value recorded by the original publication depicting P-Values, 
    Confidence Score, SGA Score, etc. Will be “-” if no score is reported. """ 
biogrid['Score'] = pd.to_numeric(biogrid['Score'])
biogrid['Score'] = biogrid['Score'].apply(lambda x: 0 if x < 0 else x)
biogrid.reset_index(drop=True, inplace=True)
biogrid.head()

In [154]:
biogrid.head()

,Official Symbol Interactor A,Official Symbol Interactor B
0,CD2BP2,SNRNP200
1,GIPC1,MYO6
2,EIF4A2,EIF4G1
3,CD2BP2,PRPF8
4,ARF6,LRPPRC


In [ ]:
""" genesA = biogrid['Official Symbol Interactor A'].tolist()
genesB = biogrid['Official Symbol Interactor B'].tolist()
genes = genesA + genesB
genes = list(dict.fromkeys(genes))
print(genes) """

In [143]:
endometriosis = pd.read_csv('genes_for_protein/ENDOMETRIOSIS.txt', sep = '\t', header=None)
endometriosis = endometriosis.rename(columns={0:'Gene'})

In [144]:

endometriosis.head()

,Gene
0,LACTBL1
1,WNT4
2,GREB1
3,RMND1
4,CCDC170


In [155]:
G = nx.Graph()

for index, row in biogrid.iterrows():
    source_node = row['Official Symbol Interactor A']
    target_node = row['Official Symbol Interactor B']
       
    G.add_edge(source_node, target_node)

In [180]:
nodes = endometriosis['Gene'].tolist()
subset_of_nodes = nodes[:10]

# Create a new graph to store the shortest paths
H = nx.Graph()

# Find the shortest path between each pair of nodes in the subset
for i in range(len(subset_of_nodes)):
    for j in range(i + 1, len(subset_of_nodes)):
        node_i = subset_of_nodes[i]
        node_j = subset_of_nodes[j]
        
        # Check if both nodes exist in graph G
        if node_i in G and node_j in G:
            try:
                # Calculate shortest path if nodes exist in G
                path = nx.shortest_path(G, node_i, node_j)
                path_edges = [(path[k], path[k+1]) for k in range(len(path)-1)]
                H.add_edges_from(path_edges)
            except nx.NetworkXNoPath:
                continue
        else:
            continue

# Find the minimum spanning tree of the new graph
mst = nx.minimum_spanning_tree(H, algorithm='kruskal')

# Print the edges of the minimum spanning tree
for edge in mst.edges(data=True):
    node1, node2 = edge[0], edge[1]
    weight = edge[2]
    print(f"Edge: {node1} -- {node2}")


Edge: WNT4 -- ITPA
Edge: WNT4 -- WLS
Edge: WNT4 -- HSPA5
Edge: WNT4 -- TOMM40
Edge: WNT4 -- CAMK2D
Edge: WNT4 -- BORA
Edge: ITPA -- TKT
Edge: TKT -- GREB1
Edge: TKT -- SDHA
Edge: TKT -- CD3EAP
Edge: GREB1 -- DNAJC7
Edge: GREB1 -- PNKD
Edge: GREB1 -- PTGES3
Edge: WLS -- nsp6
Edge: nsp6 -- RMND1
Edge: nsp6 -- AP3M1
Edge: RMND1 -- SAMM50
Edge: RMND1 -- DDX39A
Edge: RMND1 -- NDUFS1
Edge: HSPA5 -- RMDN3
Edge: HSPA5 -- SEC63
Edge: RMDN3 -- MIS18A
Edge: MIS18A -- CCDC170
Edge: MIS18A -- PCNT
Edge: MIS18A -- TMEM150A
Edge: MIS18A -- NCAPH2
Edge: MIS18A -- CENPA
Edge: MIS18A -- SENP5
Edge: TOMM40 -- FAF2
Edge: FAF2 -- ARL14EP
Edge: FAF2 -- RAB7A
Edge: DDX39A -- HOXB2
Edge: DDX39A -- SENP1
Edge: HOXB2 -- ZYG11A
Edge: BORA -- FGFR3
Edge: FGFR3 -- SNX11
Edge: SNX11 -- IFITM1
Edge: SNX11 -- PML
Edge: SNX11 -- STK40
Edge: SNX11 -- AP3B1
Edge: SEC63 -- SKAP1
Edge: SKAP1 -- SETX
Edge: ZYG11A -- ATF6


In [181]:

# Extract node positions for plotting
pos = nx.spring_layout(mst, seed=42)

# Create Plotly figure for interactive visualization
fig = go.Figure()

# Add nodes to the figure
for node in mst.nodes:
    x, y = pos[node]
    fig.add_trace(go.Scatter(x=[x], y=[y], mode='markers+text', marker=dict(size=30), text=node, textposition="middle center", name=node))

# Add edges of MST to the figure
for edge in mst.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    fig.add_trace(go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines', line=dict(width=2, color='red'), name=f"{edge[0]} - {edge[1]}"))

# Update layout options
fig.update_layout(
    title='Minimum Spanning Tree Visualization',
    showlegend=False,
    hovermode='closest',
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

# Show the interactive plot
fig.show()
